In [1]:
import os
import json
import pandas as pd
import traceback

In [ ]:
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

In [ ]:
KEY=os.getenv("OPENAI_API_KEY")

In [16]:
llm=ChatOpenAI(openai_api_key=KEY, model_name="gpt-3.5-turbo", temperature=0.5)

In [17]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [19]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be confirming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [20]:
RESPONSE_JSON={
  "1": {
    "mcq": "multiple choice question",
    "options": {
      "a": "choice here",
      "b": "choice here",
      "c": "choice here",
      "d": "choice here",
    },
    "correct": "correct answer"
  },
  "2": {
    "mcq": "multiple choice question",
    "options": {
      "a": "choice here",
      "b": "choice here",
      "c": "choice here",
      "d": "choice here",
    },
    "correct": "correct answer"
  },
  "3": {
    "mcq": "multiple choice question",
    "options": {
      "a": "choice here",
      "b": "choice here",
      "c": "choice here",
      "d": "choice here",
    },
    "correct": "correct answer"
  },
}

In [21]:
quiz_generation_prompt = PromptTemplate(
  input_variables=['text', 'number', 'subject', 'tone', 'response_json'],
  template=TEMPLATE
)

In [22]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [36]:
TEMPLATE2="""
You are an expert English grammarian and writer. Given a multiple choice quiz for {subject} students, \
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use \
50 words max for complexity. If the quiz is not at par with the cognitive and analytical abilities of the \
students, update the quiz questions which need to be changed and change the tone so it perfectly fits the students.
Make changes so there is one difficult question.

Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [37]:
quiz_evaluation_prompt=PromptTemplate(input_variables=['subject', 'quiz'], template=TEMPLATE2)

In [46]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_keys=["revised_quiz"], verbose=True)

ValidationError: 1 validation error for LLMChain
output_keys
  extra fields not permitted (type=value_error.extra)

In [39]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=['text', 'number', 'subject', 'tone', 'response_json'], output_variables=['quiz', 'revised_quiz'], verbose=True)

In [40]:
file_path="/Users/jing/mcqgen/data.txt"

In [41]:
with open(file_path,'r') as file:
  TEXT=file.read()

In [42]:
NUMBER=5
SUBJECT="machine learning"
TONE="simple"

In [43]:
with get_openai_callback() as cb:
  response=generate_evaluate_chain(
    {
      "text": TEXT,
      "number": NUMBER,
      "subject": SUBJECT,
      "tone": TONE,
      "response_json": json.dumps(RESPONSE_JSON)
    }
  )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used b

In [44]:
response

{'text': 'The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]\n\nAlthough the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb\'s model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[12] Other researchers who have studied human cognitive systems